In [2]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os
import getpass

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re
import datetime as datetime
# Ensure all necessary packages are installed
%pip install kneed

user = getpass.getuser()

if user == "JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

elif user == "aamnasoniwala":
    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

else:
    raise ValueError("Unrecognized user")

excel_path = path_equity_precision_llm_folder + "/llm training/epldat03_Unattributable Data.csv"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
api_key_epl_shared = ""

from openai import OpenAI
# https://stackoverflow.com/questions/36959031/how-to-source-file-into-python-script
execfile(path_equity_precision_llm_repo + "/constants.py")

In [3]:
execfile(path_equity_precision_llm_repo + "/functions/prompt_generator.py")
execfile(path_equity_precision_llm_repo + "/functions/base_prompt_append.py")

base_prompt_files = ['p1v6', 'p2v6', 'p3v6','p4v6']
base_prompts = base_prompt_append(base_prompt_files)

In [4]:
pmid_list = pd.read_csv(excel_path)['PMID'].tolist()

json_list_unattributable = []

# Step 1: Read the excel sheet
if excel_path.endswith('.xlsx') or excel_path.endswith('.xls'):
    df = pd.read_excel(excel_path, sheet_name= "Sheet1")
elif excel_path.endswith('.csv'):
    df = pd.read_csv(excel_path,sep=',')
else:
    raise ValueError("Unsupported file format. Please provide an Excel or CSV file.")

for index, pmid in enumerate(pmid_list):
    print(pmid)
    prompt_pmid = prompt_generator_v2(pmid, base_prompts, df)
    dict_pmid_unattributable = {"custom_id": str(index) + "_" + str(pmid), 
                 "method": "POST", 
                 "url": "/v1/chat/completions", 
                 "body": {"model": "gpt-4o-2024-08-06", 
                          "messages": [ {"role": "system", "content": base_prompts[0]},
                                        {"role": "system", "content": base_prompts[1]},
                                        {"role": "user", "content": prompt_pmid},
                                        {"role": "user", "content": base_prompts[3]}],
                            "max_tokens": 2000,
                            "frequency_penalty": -0.2,
                            "top_p": 0.8,
                            "presence_penalty": -0.3

                        }
                }

    json_list_unattributable.append(dict_pmid_unattributable)

18008027
18205795
18378359
18626579
18651247
18800199
18809217
18817993
18840627
18922591
18927166
18955282
18955324
18955365
18998207
19001480
19019358
19051287
19056300
19067246
19108908
19119030
19147138
19147525
19157583
19157823
19171337
19171400
19174316
19186040
19186042
19195861
19195867
19208721
19214789
19217093
19217176
19217229
19218986
19221398
19222503
19223085
19228266
19231259
19233878
19235213
19237208
19240085
19249904
19261277
19268929
19269155
19286335
19304868
19344959
19361872
19361966
19368908
19369060
19373130
19382911
19395095
19404781
19410444
19415164
19423318
19424668
19427818
19432816
19439916
19444228
19455018
19457510
19459046
19459059
19459835
19460054
19473183
19473824
19474798
19477539
19481430
19485923
19498343
19498344
19501492
19502018
19503022
19507060
19508213
19508241
19521345
19525080
19531084
19533015
19537931
19540739
19543209
19545916
19546874
19548094
19548280
19553094
19553142
19556978
19566830
19568747
19570586
19570664
19570670
19573144
1

In [ ]:
# Remove everything inside the folder 'Test Data Splits'


for filename in os.listdir(path_equity_precision_llm_folder + "/llm training/Unclassified Splits"):
    os.remove(path_equity_precision_llm_folder + "/llm training/Unclassified Splits" +"/" + filename)



In [6]:
# Number of splits
n_json_splits = 10
import json

# Split the JSON data
split_index = len(json_list_unattributable) // n_json_splits # n_json_splits to be specified above

In [ ]:
# Save the main chunks of the JSON data

for i in range(n_json_splits):
    if(i == n_json_splits-1):
        part = json_list_unattributable[i*split_index:]
    else:
        part = json_list_unattributable[i*split_index:(i+1)*split_index]

    with(open(path_equity_precision_llm_folder + '/llm training/Unclassified Splits/' + f"Unattributable_part{i+1}.jsonl", 'w')) as f:
        for entry in part:
            json_line = json.dumps(entry)
            f.write(json_line + '\n')
    print(f"JSON file saved successfully to OneDrive folder:" + f"Unattributable_part{i+1}.jsonl")




In [7]:
# Save remainder of the JSON data
part = json_list_unattributable[(n_json_splits)*split_index:]
with(open(path_equity_precision_llm_folder + '/llm training/Unclassified Splits/' + f"Unattributable_part{n_json_splits+1}.jsonl", 'w')) as f:
    for entry in part:
        json_line = json.dumps(entry)
        f.write(json_line + '\n')
print(f"JSON file saved successfully to OneDrive folder:" + f"Unattributable_part{n_json_splits+1}.jsonl")

JSON file saved successfully to OneDrive folder:Unattributable_part11.jsonl


In [ ]:
client = OpenAI(api_key= api_key_epl_shared)

inputs_file_path = path_equity_precision_llm_repo + "/Inputs.txt"

if not os.path.exists(inputs_file_path):
    with open(inputs_file_path, "w") as f:
        pass


with open(inputs_file_path, "a") as f:
    f.write("\n")
    current_time = datetime.datetime.now()
    f.write(f"# {current_time}")
    f.write("\n")

for i in range(n_json_splits+1):
    batch_input_file_unattributable_part1 = client.files.create(
      file = open(path_equity_precision_llm_folder + '/llm training/Unclassified Splits/' + f"Unattributable_part{i+1}.jsonl","rb"),
      purpose="batch"
    )
    
    with open(inputs_file_path, "a") as f:
      f.write(f"batch_input_file_unattributable_part{i+1} = '{batch_input_file_unattributable_part1.id}'\n")
    








In [10]:
# Enter the batch_input_file_test_partI_id from the Inputs.txt file
batch_input_file_unattributable_part1_id = "file-LdbqnEpFVYsPxGkLbYedjA"
part = 11 # This needs to be changed to the part number


print(batch_input_file_unattributable_part1_id)
batch_created_unattributable_part1 = client.batches.create(
    input_file_id=batch_input_file_unattributable_part1_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "unattributable data for PMID query - scenario 4 part 11" + str(part)
    }
)

batches_file_path = path_equity_precision_llm_repo + "/Batches.txt"

if not os.path.exists(batches_file_path):
    with open(batches_file_path, "w") as f:
        pass

with open(batches_file_path, "a") as f:
    f.write("\n")
    current_time = datetime.datetime.now()
    f.write(f"# {current_time}\n")
    f.write(f"batch_created_unattributable_part{part}_id = '{batch_created_unattributable_part1.id}'\n")


file-LdbqnEpFVYsPxGkLbYedjA


In [3]:
# batch_created_test_partI_id = batch_created_test_partI.id
batch_created_unattributable_part11_id = 'batch_67bbdc46dc8c8190a3422f5c0a9538ff'
part = 11 # This needs to be changed to the part number
client = OpenAI(api_key= api_key_epl_shared)


batch_status_unattributable_part11 = client.batches.retrieve(batch_created_unattributable_part11_id)

print("Scenario 4: " + batch_status_unattributable_part11.status)


Scenario 4: completed


In [5]:
batch_created_unattributable_part1_id = 'batch_67bbbb5ecd4c8190829b331a36abdb6b'
batch_created_unattributable_part2_id = 'batch_67bbbbdf4a4881909ae1bece1e50585c'
batch_created_unattributable_part3_id = 'batch_67bbbc44a08c81908c4eedef54114eb3'
batch_created_unattributable_part4_id = 'batch_67bbbc72257081909c0cf261bfd569b5'
batch_created_unattributable_part5_id = 'batch_67bbbcc19128819089b06b0773b688b6'
batch_created_unattributable_part6_id = 'batch_67bbbcf2482c8190916f3188d4f73d82'
batch_created_unattributable_part7_id = 'batch_67bbc140a02c8190928385bfe7f6d242'
batch_created_unattributable_part8_id = 'batch_67bbc1633adc819095016a10442b8500'
batch_created_unattributable_part9_id = 'batch_67bbc18221d881909b5d77d86fd702f9'
batch_created_unattributable_part10_id = 'batch_67bbc1a380508190bc82253e4d74bd04'
batch_created_unattributable_part11_id = 'batch_67bbdc46dc8c8190a3422f5c0a9538ff'

client = OpenAI(api_key= api_key_epl_shared)

# file_response_unattributable_part1 = client.files.content(batch_status_unattributable_part1.output_file_id)
# file_response_unattributable_part2 = client.files.content(batch_status_unattributable_part2.output_file_id)
# file_response_unattributable_part3 = client.files.content(batch_status_unattributable_part3.output_file_id)
# file_response_unattributable_part4 = client.files.content(batch_status_unattributable_part4.output_file_id)
# file_response_unattributable_part5 = client.files.content(batch_status_unattributable_part5.output_file_id)
# file_response_unattributable_part6 = client.files.content(batch_status_unattributable_part6.output_file_id)
# file_response_unattributable_part7 = client.files.content(batch_status_unattributable_part7.output_file_id)
# file_response_unattributable_part8 = client.files.content(batch_status_unattributable_part8.output_file_id)
# file_response_unattributable_part9 = client.files.content(batch_status_unattributable_part9.output_file_id)
# file_response_unattributable_part10 = client.files.content(batch_status_unattributable_part10.output_file_id)
# file_response_unattributable_part11 = client.files.content(batch_status_unattributable_part11.output_file_id)

In [6]:
import re
execfile(path_equity_precision_llm_repo + "/functions/format_gpt_output.py")

def save_results(part_number, batch_created_id):
	results = pd.DataFrame()
	batch_status = client.batches.retrieve(batch_created_id)
	print(batch_status.id)
	df = pd.read_json(path_equity_precision_llm_folder + "/llm training/Unclassified Splits/" + str(batch_status.id) +"_output.jsonl",lines=True)
	
	results = format_gpt_output_v2(df)
	results.to_csv(path_equity_precision_llm_folder + f'/llm training/Unclassified Splits/Unattributable Part {part_number}_results.csv', index=False)
	print(f"Results for part {part_number} saved successfully.")
	return results
	
save_results(1, batch_created_unattributable_part1_id)
save_results(2, batch_created_unattributable_part2_id)
save_results(3, batch_created_unattributable_part3_id)
save_results(4, batch_created_unattributable_part4_id)
save_results(5, batch_created_unattributable_part5_id)
save_results(6, batch_created_unattributable_part6_id)
save_results(7, batch_created_unattributable_part7_id)
save_results(8, batch_created_unattributable_part8_id)
save_results(9, batch_created_unattributable_part9_id)
save_results(10, batch_created_unattributable_part10_id)
save_results(11, batch_created_unattributable_part11_id)

batch_67bbbb5ecd4c8190829b331a36abdb6b


TypeError: argument of type 'method' is not iterable

In [ ]:
import re
execfile(path_equity_precision_llm_repo + "/functions/format_gpt_output.py")

results = pd.DataFrame()
batch_status = client.batches.retrieve(batch_created_unattributable_part1_id)
df = pd.read_json(path_equity_precision_llm_folder + "/llm training/Unclassified Splits/" + batch_created_unattributable_part1_id +"_output.jsonl", lines=True)


df.head()


,id,custom_id,response,error
0,batch_req_67bbd2804b58819088551989af257218,18008027,"{'status_code': 200, 'request_id': 'cf663bdf88...",NaN
1,batch_req_67bbd28062cc8190b1acefbf9793b9c8,118205795,"{'status_code': 200, 'request_id': '61f3181a06...",NaN
2,batch_req_67bbd280765c8190bea9814f890ede32,218378359,"{'status_code': 200, 'request_id': 'a824a85d2e...",NaN
3,batch_req_67bbd2808e24819083c43c03d92a067c,318626579,"{'status_code': 200, 'request_id': '762c2b7297...",NaN
4,batch_req_67bbd2809e8881909c66343fb14e29ab,418651247,"{'status_code': 200, 'request_id': 'fef9d3a877...",NaN


In [ ]:
# df = pd.read_json(file_response.content.decode('utf-8'), lines=True)
for i in range(len(df)):
	print(i)
	try:   
		markdown_table = df['response'][i]['body']['choices'][0]['message']['content'] # Extracts markdown table
		cleaned_table = re.sub(r'^.*?(?=\| PMID)', '', markdown_table, flags=re.DOTALL) # Identifies and removes everything before 'PMID' 
		table_data = cleaned_table.split('\n\n')[0]  # Extract the first part after splitting on \n\n  
		out = pd.read_csv(pd.io.common.StringIO(table_data), 
                      sep="|", 
                      skipinitialspace=False, 
                      engine='python', 
                      header=0)
		out.columns = out.columns.str.lower().str.strip().str.replace('**', '')
		results = pd.concat([results, out.iloc[[1]]], ignore_index=True)
	finally:
		continue
	
	
